# AI Orchestrator - RAG Basic

user query -> search -> generate answer

## Prep

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

> update the name of `environment variables, SC_AOAI_KEY` and the value of `index_name`

In [2]:
import os
from openai import AzureOpenAI

MODEL_GPT4 = "gpt-4-turbo"
MODEL_GPT35 = "gpt-35-turbo-1106"
client = AzureOpenAI(
    api_key=os.environ['SC_AOAI_KEY'],
    api_version="2023-12-01-preview",
    azure_endpoint = os.environ['SC_AOAI_ENDPOINT']
)

In [3]:
def _chat_gpt(messages, model=MODEL_GPT35, temp=0, topp=0.1):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp,
        max_tokens=4000,
        top_p=topp
    )   
    
    return response.choices[0].message.content

In [4]:
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

# Variables not used here do not need to be updated in your .env file
endpoint = os.environ["AZSCH_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZSCH_KEY"])

index_name = 'aoai-docs-idx'

## Search query

In [5]:
query = "What is OpenAI assistant api?"  
chat_history = []

## Search

Vector Search
- vectorquery = "query"
- search_text = None

In [8]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery

topK = 3

# Pure Vector Search
search_client = SearchClient(endpoint, index_name, credential=credential)
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=topK, fields="vector", exhaustive=True)
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["parent_id", "chunk_id", "chunk"],
    top=topK
)  
  
for result in results:  
    print(f"parent_id: {result['parent_id']}")  
    print(f"  chunk_id: {result['chunk_id']}")  
    print(f"  Score: {result['@search.score']}")  
    #print(f"Content: {result['chunk']}")

parent_id: aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2Fzc2lzdGFudHMubWQ1
  chunk_id: 08a98f1ccdb1_aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2Fzc2lzdGFudHMubWQ1_pages_0
  Score: 0.8802705
parent_id: aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2NoYXRncHQtc3R1ZGlvLm1k0
  chunk_id: b1abae7a10d7_aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2NoYXRncHQtc3R1ZGlvLm1k0_pages_1
  Score: 0.86489004
parent_id: aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2NoYXQtbWFya3VwLWxhbmd1YWdlLm1k0
  chunk_id: e993073ba3fe_aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2NoYXQtbWFya3VwLWxhbmd1YWdlLm1k0_pages_5
  Score: 0.86298484


In [9]:
def get_context(text):
    vector_query = VectorizableTextQuery(text=text, k_nearest_neighbors=topK, fields="vector", exhaustive=True)
    results = search_client.search(  
        search_text=None,  
        vector_queries= [vector_query],
        select=["parent_id", "chunk_id", "chunk", "title"],
        top=topK
    )  

    context = ""
    metadata = []
    for result in results:  
        context = context + result['chunk'] + "\n\n"
        metadata.append({"score": result['@search.score'], "chunk_id": result['chunk_id'], "parent_id": result['parent_id'], "title": result["title"]})

    return context, metadata

In [10]:
context, metadata = get_context(query)

## Response

In [11]:
# prompt templates
from jinja2 import Template
from common import parse_chat

with open('./reply_simple.jinja2') as file:
    simple_response_template = file.read()

with open('./reply_simple_ground.jinja2') as file:
    ground_response_template = file.read()

In [12]:
prompt = Template(simple_response_template, trim_blocks=True, keep_trailing_newline=True).render(
    context=context,
    chat_history=chat_history,
    user_query=query
)

print(prompt)

system:
You are a helpful and friendly assistant. Generate answer based on the context provided below. 
If you user asked in Korean then answer back in Korean otherwise answer in English only.

## Context
---
title: Azure OpenAI Service Assistant API concepts
titleSuffix: Azure OpenAI Service
description: Learn about the concepts behind the Azure OpenAI Assistants API.
ms.topic: conceptual
ms.date: 02/05/2023
manager: nitinme
author: mrbullwinkle
ms.author: mbullwin
recommendations: false
---

# Azure OpenAI Assistants API (Preview)

Assistants, a new feature of Azure OpenAI Service, is now available in public preview. Assistants API makes it easier for developers to create applications with sophisticated copilot-like experiences that can sift through data, suggest solutions, and automate tasks.

## Overview

Previously, building custom AI assistants needed heavy lifting even for experienced developers. While the chat completions API is lightweight and powerful, it's inherently statele

In [13]:
def get_response(query, chat_history, template, model=MODEL_GPT35):

    context, metadata = get_context(query)
    
    prompt = Template(template, trim_blocks=True, keep_trailing_newline=True).render(
        context=context,
        chat_history=chat_history,
        user_query=query
    )
    
    messages = parse_chat(prompt)
    
    return _chat_gpt(messages, model), metadata

## Very Simple Prompt

In [14]:
chat_history = []
query = "What is OpenAI assistant api?"  
response, metadata = get_response(query, chat_history, simple_response_template)
chat_history.append([query, response])
print(query, "\n> ", response)

What is OpenAI assistant api? 
>  The OpenAI Assistants API is a feature of the Azure OpenAI Service that allows developers to create applications with sophisticated copilot-like experiences. It enables developers to build custom AI assistants with capabilities such as sifting through data, suggesting solutions, and automating tasks. The API supports persistent automatically managed threads, allowing developers to focus on building conversational experiences without having to manage conversation state and chat threads manually. It also provides access to multiple tools, such as code interpreter and function calling, and is built on the same capabilities that power OpenAI’s GPT product.


In [15]:
metadata

[{'score': 0.8802705,
  'chunk_id': '08a98f1ccdb1_aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2Fzc2lzdGFudHMubWQ1_pages_0',
  'parent_id': 'aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2Fzc2lzdGFudHMubWQ1',
  'title': 'assistants.md'},
 {'score': 0.86489004,
  'chunk_id': 'b1abae7a10d7_aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2NoYXRncHQtc3R1ZGlvLm1k0_pages_1',
  'parent_id': 'aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2NoYXRncHQtc3R1ZGlvLm1k0',
  'title': 'chatgpt-studio.md'},
 {'score': 0.86298484,
  'chunk_id': 'e993073ba3fe_aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2NoYXQtbWFya3VwLWxhbmd1YWdlLm1k0_pages_5',
  'parent_id': 'aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2NoYXQtbWFya3VwLWxhbmd1YWdlLm1k0',
  'title': 'chat-markup-language.md'}]

In [17]:
query = "What is Google"
response, metadata = get_response(query, chat_history, simple_response_template)
chat_history.append([query, response])
print(query, "\n> ", response)

What is Google 
>  Google is a multinational technology company that specializes in Internet-related services and products. It was founded in 1998 by Larry Page and Sergey Brin while they were Ph.D. students at Stanford University. Google's services include online advertising technologies, a search engine, cloud computing, software, and hardware. The company is known for its search engine, also called Google, which is the most widely used search engine on the World Wide Web.


In [18]:
metadata

[{'score': 0.79577667,
  'chunk_id': 'c617f41e61bc_aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2NvbXBsZXRpb25zLm1k0_pages_15',
  'parent_id': 'aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL2NvbXBsZXRpb25zLm1k0',
  'title': 'completions.md'},
 {'score': 0.78871673,
  'chunk_id': '8f8b94abc7cf_aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL3Byb21wdC1jaGF0LWNvbXBsZXRpb24ubWQ1_pages_18',
  'parent_id': 'aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL3Byb21wdC1jaGF0LWNvbXBsZXRpb24ubWQ1',
  'title': 'prompt-chat-completion.md'},
 {'score': 0.7847235,
  'chunk_id': '63407be9297e_aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL3VzZS15b3VyLWltYWdlLWRhdGEubWQ1_pages_7',
  'parent_id': 'aHR0cHM6Ly9pa2Jsb2JhY2N0LmJsb2IuY29yZS53aW5kb3dzLm5ldC9kb2NzL3VzZS15b3VyLWltYWdlLWRhdGEubWQ1',
  'title': 'use-your-image-data.md'}]

## Grounded Prompt

ground response using `ground_response_template`

In [21]:
chat_history = []
query = "What is OpenAI assistant api?"  
response, metadata = get_response(query, chat_history, ground_response_template)
chat_history.append([query, response])
print(query, "\n> ", response)

What is OpenAI assistant api? 
>  The Assistants API, as the stateful evolution of the chat completion API, provides a solution for challenges faced by developers. It supports persistent automatically managed threads, which means that developers no longer need to develop conversation state management systems and work around a model’s context window constraints. Once you create a Thread, you can simply append new messages to it as users respond. Assistants can also access multiple tools in parallel, if needed.


In [20]:
query = "What is Google"
response, metadata = get_response(query, chat_history, ground_response_template)
chat_history.append([query, response])
print(query, "\n> ", response)

What is Google 
>  Sorry, I don't know.


## Condensed Question

example of condense question:

- first question "what is assistant api?"
- second question "show sample code in python"

intent of second question is "show sample code for assistant api in python"

In [24]:
chat_history = []
query = "What is OpenAI assistant api?"  
response, metadata = get_response(query, chat_history, ground_response_template)
chat_history.append([query, response])
print(query, "\n> ", response)

What is OpenAI assistant api? 
>  The Assistants API, as the stateful evolution of the chat completion API, provides a solution for challenges faced by developers. It supports persistent automatically managed threads, which means that developers no longer need to develop conversation state management systems and work around a model’s context window constraints. Once you create a Thread, you can simply append new messages to it as users respond. Assistants can also access multiple tools in parallel, if needed.


> unrelated response by retrieving wrong documents

In [25]:
query = "show sample code in python"
response, metadata = get_response(query, chat_history, ground_response_template)
chat_history.append([query, response])
print(query, "\n> ", response)

show sample code in python 
>  ```python
# Python 3
# Calculate the mean distance between an array of points
```


In [26]:
query = "show OpenAI assistant api sample code in python"
response, metadata = get_response(query, chat_history, ground_response_template)
chat_history.append([query, response])
print(query, "\n> ", response)

show OpenAI assistant api sample code in python 
>  To use the OpenAI Service via the Python SDK, you can create an assistant with the following code:

```python
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

assistant = client.beta.assistants.create(
  instructions="You are an AI assistant that can write code to help answer math questions",
  model="<REPLACE WITH MODEL DEPLOYMENT NAME>", # replace with model deployment name. 
  tools=[{"type": "code_interpreter"}]
)
```
